In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import numpy as np

In [2]:
data = pd.read_csv('Maternal Health Risk Data Set.csv')
data['RiskLevel'] = data['RiskLevel'].map({'low risk': 0, 'mid risk': 1, 'high risk': 2})
data['PulsePressure'] = data['SystolicBP'] - data['DiastolicBP']
data['MAP'] = data['DiastolicBP'] + (data['PulsePressure'] / 3)
data['BPRatio'] = data['SystolicBP'] / data['DiastolicBP']
data['SBP/Age'] = data['SystolicBP'] / data['Age']
data['DBP/Age'] = data['DiastolicBP'] / data['Age']
data['BS/Age'] = data['BS'] / data['Age']
data.drop(columns=(['HeartRate', 'DiastolicBP']), inplace=True)
x = data.drop(columns=['RiskLevel'])
y = data['RiskLevel']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=10)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [3]:
load_model = tf.keras.models.load_model('maternal_health_risk_model.keras')
y_new_pred_train = load_model.predict(x_train)
y_new_pred_test = load_model.predict(x_test)
y_new_pred_train_classes = np.argmax(y_new_pred_train, axis=1)
y_new_pred_test_classes = np.argmax(y_new_pred_test, axis=1)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [4]:
A = confusion_matrix(y_train, y_new_pred_train_classes)
num_classes = A.shape[0]

TP = np.diag(A)
FP = np.sum(A, axis=0) - TP
FN = np.sum(A, axis=1) - TP
TN = np.sum(A) - (TP + FP + FN)

# Precision: TP / (TP + FP)
precision_per_class = TP / (TP + FP + 1e-12)

# Recall: TP / (TP + FN)
recall_per_class = TP / (TP + FN + 1e-12)

# Accuracy
accuracy = np.sum(TP) / np.sum(A)

# Macro averages
precision_macro = np.mean(precision_per_class)
recall_macro = np.mean(recall_per_class)

print("\n=== TRAIN METRICS ===")
print("Accuracy:", accuracy)

print("\nPrecision per class:", precision_per_class)
print("Macro Precision:", precision_macro)

print("\nRecall per class:", recall_per_class)
print("Macro Recall:", recall_macro)


=== TRAIN METRICS ===
Accuracy: 0.875725900116144

Precision per class: [0.89253731 0.82807018 0.90871369]
Macro Precision: 0.8764403939391604

Recall per class: [0.85673352 0.83687943 0.95217391]
Macro Recall: 0.8819289566742944


In [5]:
A = confusion_matrix(y_test, y_new_pred_test_classes)
num_classes = A.shape[0]

TP = np.diag(A)
FP = np.sum(A, axis=0) - TP
FN = np.sum(A, axis=1) - TP
TN = np.sum(A) - (TP + FP + FN)

# Precision: TP / (TP + FP)
precision_per_class = TP / (TP + FP + 1e-12)

# Recall: TP / (TP + FN)
recall_per_class = TP / (TP + FN + 1e-12)

# Accuracy
accuracy = np.sum(TP) / np.sum(A)

# Macro averages
precision_macro = np.mean(precision_per_class)
recall_macro = np.mean(recall_per_class)

print("\n=== TEST METRICS ===")
print("Accuracy:", accuracy)

print("\nPrecision per class:", precision_per_class)
print("Macro Precision:", precision_macro)

print("\nRecall per class:", recall_per_class)
print("Macro Recall:", recall_macro)


=== TEST METRICS ===
Accuracy: 0.8169934640522876

Precision per class: [0.83018868 0.74137931 0.9047619 ]
Macro Precision: 0.8254432981173218

Recall per class: [0.77192982 0.7962963  0.9047619 ]
Macro Recall: 0.824329341873185
